# Exercises for Practice

## Exercise 01 

The data below come from [tidytuesday](https://github.com/rfordatascience/tidytuesday/tree/master/data/2019/2019-09-10) and provide information on accidents at theme parks. You can see more of these [data available here](https://ridesdatabase.org/saferparks/data/). The data give you some details of where and when the accident occurred, and something about the injured party as well. 

In [1]:
library(readr)

read_csv(
    "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-09-10/saferparks.csv"
    ) -> safer_parks

Rows: 8351 Columns: 23

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (17): acc_date, acc_state, acc_city, fix_port, source, bus_type, industr...
dbl  (6): acc_id, num_injured, age_youngest, mechanical, op_error, employee


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



|variable             |class     |description |
|:--------------------|:---------|:-----------|
|acc_id               |double    | Unique ID |
|acc_date             |character | Accident Date |
|acc_state            |character | Accident State |
|acc_city             |character | Accident City |
|fix_port             |character |.           |
|source               |character | Source of injury report |
|bus_type             |character | Business type |
|industry_sector      |character | Industry sector |
|device_category      |character | Device category |
|device_type          |character | Device type |
|tradename_or_generic |character | Common name of the device |
|manufacturer         |character | Manufacturer of device |
|num_injured          |double    | Num injured |
|age_youngest         |double    | Youngest individual injured |
|gender               |character | Gender of individual injured |
|acc_desc             |character | Description of accident |
|injury_desc          |character | Injury description |
|report               |character | Report URL |
|category             |character | Category of accident |
|mechanical           |double    | Mechanical failure (binary NA/1) |
|op_error             |double    | Operator error (binary NA/1)|
|employee             |double    | Employee error (binary NA/1)|
|notes                |character | Additional notes| 

Working with the `safer_parks` data, complete the following tasks. 

### Problem (a)
Using `acc_date`, create a new date variable called `idate` that is a proper date column generated via ``{lubridate}``. 

In [2]:
library(tidyverse)
library(lubridate)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ dplyr   1.0.7
✔ tibble  3.1.6     ✔ stringr 1.4.0
✔ tidyr   1.1.4     ✔ forcats 0.5.1
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [3]:
head(safer_parks)

acc_id,acc_date,acc_state,acc_city,fix_port,source,bus_type,industry_sector,device_category,device_type,⋯,age_youngest,gender,acc_desc,injury_desc,report,category,mechanical,op_error,employee,notes
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1005813,6/12/2010,OH,Cleveland,F,Ohio Dept. of Agriculture,Sports or recreation facility,recreation,inflatable,Inflatable slide,⋯,NA,NA,"Inflatable slide tipped over while 7-9 patrons on top of it. The slide fell on a man standing next to it, causing several fractures to his back, and minor injuries to several children. The accident was caused by allowing too many patrons on the device and operating it in an usafe manner. &nbsp;The insurance company had solicited volunteers from their employees to staff the device in return for free tickets to the ball game.","The man who was crushed by the device died 9 days later of complications from his injuries. 2 children were transported to the ER, and 6 children were treated at the scene.",https://saferparksdata.org/sites/default/files/accident_reports/OH-2010-06-12.pdf,"Device tipped over, blew away, or collapsed",NA,1,NA,"http://www.cleveland.com/metro/index.ssf/2012/10/cleveland_indians_settle_wrong.html href=""http://www.cleveland.com/metro/index.ssf/2012/10/cleveland_indians_settle_wrong.html"">http://www.cleveland.com/metro/index.ssf/2012/10/cleveland_indians_settle_wrong.html</a></div>"
1004032,6/12/2010,OH,Cleveland,P,United States Consumer Product Safety Commission,Sports or recreation facility,recreation,inflatable,Inflatable slide,⋯,54,M,Two adult males were standing next to an inflatable slide when it tipped over on top of them. &nbsp;One man suffered a fractured vertebrae and died later from complications. The other sustained head and back injuries. &nbsp;Six children who were on the slide sustained minor injuries. &nbsp;The investigation revealed the slide tipped over because there were too many children on one side and the slide was not properly anchored. &nbsp;Volunteers were acting as attendants for the amusement device. &nbsp;There were 7-9 children on the slide when it fell over.,"1 - death from pulmonary embolism caused by blunt force trauma; 1 - head and back injuries; 6 children had cuts, bruises, back and ear pain.",https://saferparksdata.org/sites/default/files/accident_reports/CPSC-OH-2010-06-12.pdf,"Device tipped over, blew away, or collapsed",NA,1,NA,See also OH state record.
1007658,7/10/2010,CA,Anaheim,F,California Division of Occupational Safety and Health,Amusement park,amusement ride,water ride,Boat ride,⋯,37,F,"Patron stated she slipped and fell while stepping into the boat, causing right hip and low back pain. Then made a late report stating torn meniscus in her knee.","Hip and low back pain, torn meniscus in knee",NA,Load/Unload: scrape or stumble,NA,NA,NA,NA
1007098,7/10/2010,CA,Carlsbad,F,California Division of Occupational Safety and Health,Water park,water park,float attraction,Lazy river,⋯,37,F,Patron claims strained back after her raft tipped. Husband reported the injury six months later.,Strained back,NA,"Fall: patron fell off inner tube, mat or board",NA,NA,NA,NA
1000094,7/29/2010,CO,Littleton,F,Colorado Division of Oil and Public Safety,Family entertainment center,recreation,challenge activity,Climbing wall,⋯,NA,M,"Two patrons climbed to the top of the wall, counted to three and jumped of the wall in unison. The auto-belay failed for one patron.",Patron experienced a free fall that resulted in three broken bones in foot.,https://saferparksdata.org/sites/default/files/accident_reports/CO-2010-07-29.pdf,Fall: patron fell from device (participatory),1,NA,NA,NA
1005756,7/30/2010,WI,Wisconsin Dells,F,"Wisconsin Dept. of Commerce, Safety and Building Division",Amusement park,recreation,challenge activity,Freefall,⋯,12,F,"Rider was released by the ride operator before the net was raised and airbag inflated, causing her to freefall 100 

In [4]:
safer_parks %>%
    mutate(
        idate = mdy(acc_date)
    ) -> safer_parks

head(safer_parks$idate)

[1] "2010-06-12" "2010-06-12" "2010-07-10" "2010-07-10" "2010-07-29"
[6] "2010-07-30"

### Problem (b)
Now create new columns for (i) the month of the accident, and (ii) the day of the week. These should not be abbreviated (i.e., we should see the values as 'Monday' instead of 'Mon', "July" instead of "Jul"). 

What month had the highest number of accidents? 

What day of the week had the highest number of accidents? 

In [5]:
safer_parks %>%
    mutate(
        month = month(idate, label = TRUE, abbr = FALSE),
        day = wday(idate, label = TRUE, abbr = FALSE) 
    ) -> safer_parks

In [6]:
head(safer_parks)

acc_id,acc_date,acc_state,acc_city,fix_port,source,bus_type,industry_sector,device_category,device_type,⋯,injury_desc,report,category,mechanical,op_error,employee,notes,idate,month,day
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<date>,<ord>,<ord>
1005813,6/12/2010,OH,Cleveland,F,Ohio Dept. of Agriculture,Sports or recreation facility,recreation,inflatable,Inflatable slide,⋯,"The man who was crushed by the device died 9 days later of complications from his injuries. 2 children were transported to the ER, and 6 children were treated at the scene.",https://saferparksdata.org/sites/default/files/accident_reports/OH-2010-06-12.pdf,"Device tipped over, blew away, or collapsed",NA,1,NA,"http://www.cleveland.com/metro/index.ssf/2012/10/cleveland_indians_settle_wrong.html href=""http://www.cleveland.com/metro/index.ssf/2012/10/cleveland_indians_settle_wrong.html"">http://www.cleveland.com/metro/index.ssf/2012/10/cleveland_indians_settle_wrong.html</a></div>",2010-06-12,June,Saturday
1004032,6/12/2010,OH,Cleveland,P,United States Consumer Product Safety Commission,Sports or recreation facility,recreation,inflatable,Inflatable slide,⋯,"1 - death from pulmonary embolism caused by blunt force trauma; 1 - head and back injuries; 6 children had cuts, bruises, back and ear pain.",https://saferparksdata.org/sites/default/files/accident_reports/CPSC-OH-2010-06-12.pdf,"Device tipped over, blew away, or collapsed",NA,1,NA,See also OH state record.,2010-06-12,June,Saturday
1007658,7/10/2010,CA,Anaheim,F,California Division of Occupational Safety and Health,Amusement park,amusement ride,water ride,Boat ride,⋯,"Hip and low back pain, torn meniscus in knee",NA,Load/Unload: scrape or stumble,NA,NA,NA,NA,2010-07-10,July,Saturday
1007098,7/10/2010,CA,Carlsbad,F,California Division of Occupational Safety and Health,Water park,water park,float attraction,Lazy river,⋯,Strained back,NA,"Fall: patron fell off inner tube, mat or board",NA,NA,NA,NA,2010-07-10,July,Saturday
1000094,7/29/2010,CO,Littleton,F,Colorado Division of Oil and Public Safety,Family entertainment center,recreation,challenge activity,Climbing wall,⋯,Patron experienced a free fall that resulted in three broken bones in foot.,https://saferparksdata.org/sites/default/files/accident_reports/CO-2010-07-29.pdf,Fall: patron fell from device (participatory),1,NA,NA,NA,2010-07-29,July,Thursday
1005756,7/30/2010,WI,Wisconsin Dells,F,"Wisconsin Dept. of Commerce, Safety and Building Division",Amusement park,recreation,challenge activity,Freefall,⋯,"Med-flighted to hospital: skull fracture; multiple severe fractures to back and pelvise; lacerated spleen, liver, intestines",NA,Fall: ejection/fall from ride,NA,1,NA,"https://www.cbsnews.com/news/park-ride-victims-father-she-was-dead/, http://www.nhregister.com/news/article/Miracle-girl-survives-fall-from-Terminal-11612562.php, http://www.wkow.com/global/story.asp?s=13003233 href=""https://www.cbsnews.com/news/park-ride-victims-father-she-was-dead/"">https://www.cbsnews.com/news/park-ride-victims-father-she-was-dead/</a>, <a href=""http://www.nhregister.com/news/article/Miracle-girl-survives-fall-from-Terminal-11612562.php"">http://www.nhregister.com/news/article/Miracle-girl-survives-fall-from-Terminal-11612562.php</a>, <a href=""http://www.wkow.com/global/story.asp?s=13003233"">http://www.wkow.com/global/story.asp?s=13003233</a></div>",2010-07-30,July,Friday


In [7]:
# July had the most accidents 

safer_parks %>%
    group_by(month) %>%
    tally() %>%
    arrange(-n)

month,n
<ord>,<int>
July,1702
August,1448
June,1322
May,706
September,575
March,493
April,484
October,461
December,320


In [8]:
# Saturdays had the most accidents

safer_parks %>%
    group_by(day) %>%
    tally() %>%
    arrange(-n)

day,n
<ord>,<int>
Saturday,2070
Sunday,1633
Friday,1128
Monday,985
Thursday,926
Wednesday,830
Tuesday,779


### Problem (c)
What if you look at days of the week by month? Does the same day of the week show up with the most accidents regardless of month or do we see some variation? 

In [9]:
# Saturdays are most common days for April, May, June, July, August, September, and October 

safer_parks %>%
    group_by(month, day) %>%
    tally() %>%
    arrange(-n)

month,day,n
<ord>,<ord>,<int>
July,Saturday,330
August,Saturday,328
June,Saturday,316
July,Sunday,287
August,Sunday,259
July,Friday,251
July,Thursday,218
June,Sunday,217
July,Monday,216


### Problem (d)
What were the `five` dates with the most number of accidents? 

In [10]:
safer_parks %>%
    group_by(idate) %>%
    tally() %>%
    arrange(-n) %>%
    top_n(., 10)

Selecting by n



idate,n
<date>,<int>
2014-08-16,23
2012-07-21,22
2015-07-11,21
2015-06-13,20
2015-06-14,20
2015-07-19,20
2016-08-06,20
2014-06-14,19
2015-07-13,19


### Problem (e)
Using the Texas injury data, answer the following question: What ride was the safest? [Hint: For each ride (`ride_name`) you will need to calculate the number of days between accidents. The ride with the highest number of days is the safest.] 

In [11]:
read_csv(
  "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-09-10/tx_injuries.csv"
  ) -> tx_injuries

Rows: 542 Columns: 13

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (12): name_of_operation, city, st, injury_date, ride_name, serial_no, ge...
dbl  (1): injury_report_rec


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.




|variable          |class     |description |
|:-----------------|:---------|:-----------|
|injury_report_rec |double    | Unique Record ID |
|name_of_operation |character | Company name |
|city              |character | City |
|st                |character | State (all TX) |
|injury_date       |character | Injury date - note there are some different formats |
|ride_name         |character | Ride Name |
|serial_no         |character | Serial number of ride |
|gender            |character | Gender of the injured individual |
|age               |character | Age of the injured individual |
|body_part         |character | Body part injured |
|alleged_injury    |character | Alleged injury - type of injury |
|cause_of_injury   |character | Approximate cause of the injury (free text) |
|other             |character | Anecdotal information in addition to cause of injury |

You should note that this assumes each ride was in operation for the same amount of time. If this is not true then our estimates will be unreliable. 

In [12]:
tx_injuries %>%
  mutate(date = mdy(injury_date)) %>%
  group_by(ride_name) %>%
  arrange(date) %>%
  mutate(
    tspan = interval(lag(date, order_by = ride_name), date),
    tspan.days = as.duration(tspan)/ddays(1)
  ) %>%
  select(date, ride_name, tspan, tspan.days) %>%
  arrange(-tspan.days)


Warning message:
“ 349 failed to parse.”


date,ride_name,tspan,tspan.days
<date>,<chr>,<Interval>,<dbl>
2015-04-16,iFly Austin,2013-09-14 UTC--2015-04-16 UTC,579
2014-07-26,Batman,2013-05-27 UTC--2014-07-26 UTC,425
2014-08-25,Coyote Cannon,2013-07-07 UTC--2014-08-25 UTC,414
2014-09-07,Tsunami Surge,2013-07-28 UTC--2014-09-07 UTC,406
2014-07-25,Howlin' Tornado,2013-06-30 UTC--2014-07-25 UTC,390
2014-08-31,Stingray Falls,2013-08-10 UTC--2014-08-31 UTC,386
2014-07-04,Cliffhanger,2013-07-07 UTC--2014-07-04 UTC,362
2014-05-16,Go Karts,2013-06-04 UTC--2014-05-16 UTC,346
2014-06-17,Mr. Freeze,2013-07-18 UTC--2014-06-17 UTC,334


## Exercise 02
These data (see below) come from this story: [The next generation: The space race is dominated by new contenders](https://www.economist.com/graphic-detail/2018/10/18/the-space-race-is-dominated-by-new-contenders). You have data on space missions over time, with dates of the launch, the launching agency/country, type of launch vehicle, and so on. 


| variable    | definition                               |
| ----------- | ---------------------------------------- |
| tag         | Harvard or [COSPAR][cospar] id of launch |
| JD          | [Julian Date][jd] of launch              |
| launch_date | date of launch                           |
| launch_year | year of launch                           |
| type        | type of launch vehicle                   |
| variant     | variant of launch vehicle                |
| mission     | space mission                            |
| agency      | launching agency                         |
| state_code  | launching agency's state                 |
| category    | success (O) or failure (F)               |
| agency_type | type of agency                           |

In [13]:
read_csv(
  "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-01-15/launches.csv"
  ) -> launches

Rows: 5726 Columns: 11

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): tag, type, variant, mission, agency, state_code, category, agency_...
dbl  (2): JD, launch_year
date (1): launch_date


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



### Problem (a) 
Create a new column called `date` that stores `launch_date` as a proper data field in ymd format from {lubridate}. 

In [14]:
launches %>%
  mutate(
      date = ymd(launch_date)
  ) -> lau.df

In [15]:
lau.df %>%
    select(date) %>%
    head()

date
<date>
1967-06-29
1967-08-23
1967-10-11
1968-05-23
1968-10-23
1969-07-23


### Problem (b) 
Creating columns as needed, calculate and show the number of launches first by year, then by month, and then by day of the week. The result should be arranged in descending order of the number of launches. 

In [16]:
lau.df %>%
  mutate(
    year = year(date),
    month = month(date, abbr = FALSE, label = TRUE),
    day = day(date),
    dow = wday(date, abbr = FALSE, label = TRUE)
  ) -> lau.df

In [17]:
lau.df %>%
  filter(!is.na(year)) %>%
  count(year, sort = TRUE) 

year,n
<dbl>,<int>
1967,139
1971,134
1975,132
1966,131
1976,131
1977,130
1982,129
1983,129
1984,129


In [18]:
lau.df %>%
  filter(!is.na(month)) %>%
  count(month, sort = TRUE) 

month,n
<ord>,<int>
December,579
June,524
October,523
September,511
April,504
August,495
July,466
May,460
March,447


In [19]:
lau.df %>%
  filter(!is.na(day)) %>%
  count(day, sort = TRUE) 

day,n
<int>,<int>
28,248
25,227
24,217
14,213
21,211
15,210
26,208
16,206
20,205


In [20]:
lau.df %>%
  filter(!is.na(dow)) %>%
  count(dow, sort = TRUE) 

dow,n
<ord>,<int>
Thursday,1112
Wednesday,1102
Friday,1030
Tuesday,971
Saturday,618
Monday,534
Sunday,346


### Problem (c) 
How many launches were successful `(O)` versus failed `(F)` by country and year? The countries of interest will be state_code values of "CN", "F", "J", "RU", "SU", "US". You do not need to arrange your results in any order. 

In [21]:
lau.df %>%
  filter(
    !is.na(date),
    state_code %in% c("CN", "F", "J", "RU", "SU", "US")
    ) %>%
  count(state_code, year, category)

state_code,year,category,n
<chr>,<dbl>,<chr>,<int>
CN,1970,O,1
CN,1971,O,1
CN,1973,F,1
CN,1974,F,2
CN,1975,O,3
CN,1976,F,1
CN,1976,O,2
CN,1978,O,1
CN,1979,F,1
